In [2]:
#!/usr/bin/env python3
# Copyright 2022 ETH Zurich and University of Bologna.
# Licensed under the Apache License, Version 2.0, see LICENSE for details.
# SPDX-License-Identifier: Apache-2.0

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import argparse
import pathlib
import hjson
import random
import os

In [3]:
np.random.seed(42)
torch.manual_seed(42)
global verbose

In [5]:
def array_to_cstr(a):
    out = '{'
    if isinstance(a, np.ndarray):
        a = a.flat
    if isinstance(a, torch.Tensor):
        a = a.numpy().flat
    for el in a:
        out += '{}, '.format(el)
    out = out[:-2] + '}'
    return out

In [28]:
def emit_mnist_data(name='mini_mnist', **kwargs):
    
    # constants
    IN_CH1 = kwargs['IN_CH1']
    IN_CH2 = kwargs['IN_CH2']
    OUT_CH = kwargs['OUT_CH']
    DATASET_SIZE = kwargs['DATASET_SIZE']
    
    # data
    MAT_INPUT = kwargs['INPUT']
    MAT_LABELS = kwargs['LABELS']

    # network init parameters from golden model
    MAT_WEIGHTS = kwargs['WEIGHTS']
    MAT_BIASES = kwargs['BIASES']
    MAT_WEIGHT_GRADS = kwargs['WEIGHT GRADIENTS']
    MAT_BIAS_GRADS = kwargs['BIAS GRADIENTS'] 

    IN_CH = IN_CH1*IN_CH2
    
    layer_str = ''
    layer_str += '#include "network.h"\n\n'
    layer_str += f'network_t {name}_t = {{\n'
    layer_str += f'\t.IN_CH1 = {IN_CH1},\n'
    layer_str += f'\t.IN_CH2 = {IN_CH2},\n'
    layer_str += f'\t.OUT_CH = {OUT_CH},\n'
    layer_str += f'\t.dtype = FP{kwargs["prec"]}\n'
    layer_str += '};\n\n\n'

    ctypes = {
        '64': 'double',
        '32': 'float',
        '16': '__fp16',
        'B16': '__bf16',
        '8': 'char'
    }

    dtype = ctypes[str(kwargs['prec'])]

    # network initialization
    layer_str += f'static {dtype} {name}_weights_dram [{OUT_CH}][{IN_CH}] = ' + array_to_cstr(MAT_WEIGHTS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_biases_dram [{OUT_CH}][{1}] = ' + array_to_cstr(MAT_BIASES) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_weight_grads_dram [{OUT_CH}][{IN_CH}] = ' + array_to_cstr(MAT_WEIGHT_GRADS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_bias_grads_dram [{OUT_CH}][{1}] = ' + array_to_cstr(MAT_BIAS_GRADS) + ';\n\n\n'


    # input data
    #layer_str += f'static {dtype} {name}_images_dram [{DATASET_SIZE*IN_CH}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'
    #layer_str += f'static uint32_t {name}_labels_dram [{DATASET_SIZE}][{1}] = ' + array_to_cstr(MAT_LABELS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_images_dram [{IN_CH}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'
    layer_str += f'static uint32_t {name}_labels_dram[{1}] = ' + array_to_cstr(MAT_LABELS) + ';\n\n\n'

    return layer_str


In [29]:
def emit_mnist_header_file(layer_type: str, **kwargs):

    file_path = '/home/msc22f11/snitch/sw/applications/data/'
    emit_str = "// Copyright 2022 ETH Zurich and University of Bologna.\n" + \
               "// Licensed under the Apache License, Version 2.0, see LICENSE for details.\n" + \
               "// SPDX-License-Identifier: Apache-2.0\n\n"

    if(layer_type == 'mini_mnist'):
        file = file_path + 'data_mini_mnist.h'
        emit_str += emit_mnist_data(**kwargs)

    with open(file, 'w') as f:
        f.write(emit_str)


In [4]:
# download MNIST dataset using DataLoader

transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_dataset = MNIST(PATH_DATASETS, train=True, transform=transform, download=True)

# set seeds for reproducability 
g = torch.Generator()
g.manual_seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

mnist_dl = DataLoader(mnist_dataset, worker_init_fn=seed_worker, generator=g)

In [41]:
"""
Now we iterate through the dataset 
to retrieve the image data with their
respective labels
"""

data_iterator = iter(mnist_dl)

for i in range(0, len(mnist_dl)):
    image, label = data_iterator.next()
    np_image = image.numpy().flatten()
    np_label = label.numpy().flatten()
    if(i==0):
        images = np.array(np_image.tolist())
        labels = np.array(np_label.tolist())
    else:
        images = np.append(images, np_image)
        labels = np.append(labels, np_label)

In [17]:
"""
Now we iterate through a smaller subset of the dataset 
to retrieve the image data with their
respective labels
"""

data_iterator = iter(mnist_dl)

for i in range(0, 1):
    s_image, s_label = data_iterator.next()
    np_s_image = s_image.numpy().flatten()
    np_s_label = s_label.numpy().flatten()
    if(i==0):
        s_images = np.array(np_s_image.tolist())
        s_labels = np.array(np_s_label.tolist())
    else:
        s_images = np.append(s_images, np_s_image)
        s_labels = np.append(s_labels, np_s_label)

In [5]:
in_ch = 28*28
out_ch = 10

class LinLayer(nn.Module):
    def __init__(self):
        super(LinLayer, self).__init__()
        torch.manual_seed(42)
        self.lin = nn.Linear(in_ch, out_ch)

    def forward(self, x):
        torch.manual_seed(42)
        out = self.lin(x.view(x.size(0), -1))
        return out

In [6]:
first_im, first_label = next(iter(mnist_dl))

In [7]:
torch.manual_seed(42)

net = LinLayer()
weights = net.lin.weight
biases = net.lin.bias

criterion = nn.CrossEntropyLoss()

for i in range(1):
    net.zero_grad
    output = net(first_im)
    loss = criterion(output, first_label)
    loss.backward()
    weight_grads = net.lin.weight.grad
    bias_grads = net.lin.bias.grad


In [18]:
kwargs = {
            'IN_CH1': 28,
            'IN_CH2': 28,
            'OUT_CH': 10,
            'DATASET_SIZE': 256,
            'INPUT': s_images,
            'LABELS': s_labels,
            'WEIGHTS': weights.detach(),
            'BIASES': biases.detach(),
            'WEIGHT GRADIENTS': weight_grads,
            'BIAS GRADIENTS': bias_grads,
            'prec': 64
}

In [30]:
emit_mnist_header_file('mini_mnist', **kwargs)

In [8]:
print(output)
sm = nn.Softmax(dim=1)
sm(output)

tensor([[-0.1598, -0.0484,  0.0685,  0.4418,  0.0252, -0.2573,  0.2446, -0.1718,
          0.2287,  0.0333]], grad_fn=<AddmmBackward0>)


tensor([[0.0801, 0.0895, 0.1006, 0.1462, 0.0964, 0.0727, 0.1200, 0.0791, 0.1181,
         0.0972]], grad_fn=<SoftmaxBackward0>)

In [36]:
torch.max(output)

tensor(0.4418, grad_fn=<MaxBackward1>)

In [26]:
weights[0]

tensor([ 2.7305e-02,  2.9643e-02, -8.3669e-03,  3.2808e-02, -7.8251e-03,
         7.2068e-03, -1.7388e-02,  2.0974e-02,  3.1484e-02, -2.6201e-02,
         3.1043e-02,  6.6843e-03,  2.6386e-02,  4.8368e-03,  1.7221e-02,
        -5.0425e-03,  2.7532e-02,  5.2789e-03, -1.6673e-02,  9.1035e-03,
        -1.6455e-02, -4.1883e-03, -1.4506e-02,  2.3692e-02, -2.8192e-02,
        -1.6465e-02, -1.0085e-02, -2.1474e-02,  3.3708e-03, -3.5274e-02,
         3.2254e-02, -3.0338e-02,  2.7572e-02,  5.9435e-03, -1.1597e-02,
         2.2070e-02,  5.5661e-03,  2.8856e-02,  3.9043e-03, -1.1263e-02,
         9.5958e-03, -9.6850e-03,  1.5031e-02,  3.1887e-02,  2.0645e-02,
        -1.5613e-02,  2.0617e-02,  6.3902e-03,  1.8137e-02, -2.1768e-02,
        -3.5354e-02, -1.3799e-02, -2.7394e-02,  2.9305e-02,  1.0287e-02,
         1.4793e-02,  1.1295e-02, -6.2128e-04,  2.7950e-02, -2.5375e-02,
         2.2487e-03, -2.4376e-02,  1.1013e-02, -1.2299e-02,  1.0943e-02,
        -7.4408e-03,  2.9621e-02, -2.1168e-02, -2.1

In [25]:
s_image.flatten().shape

torch.Size([784])

In [32]:
s_image.flatten().numpy()[153]

0.07058824

In [31]:
np.where(s_image.flatten().numpy()!= 0)

(array([152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 176,
        177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189,
        190, 191, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 231, 232, 233, 234, 235, 236, 237, 238,
        239, 240, 241, 260, 261, 262, 263, 264, 265, 266, 268, 269, 289,
        290, 291, 292, 293, 319, 320, 321, 322, 347, 348, 349, 350, 376,
        377, 378, 379, 380, 381, 405, 406, 407, 408, 409, 410, 434, 435,
        436, 437, 438, 439, 463, 464, 465, 466, 467, 493, 494, 495, 496,
        518, 519, 520, 521, 522, 523, 524, 544, 545, 546, 547, 548, 549,
        550, 551, 570, 571, 572, 573, 574, 575, 576, 577, 578, 596, 597,
        598, 599, 600, 601, 602, 603, 604, 605, 622, 623, 624, 625, 626,
        627, 628, 629, 630, 631, 648, 649, 650, 651, 652, 653, 654, 655,
        656, 657, 676, 677, 678, 679, 680, 681, 682, 683]),)

In [ ]:
weights[0]*s_image.flatten()

: 

In [1]:
import pycparser as pyc
from pyclibrary import CParser


parser = CParser('/home/msc22f11/snitch/sw/applications/data/data_full_mnist.h')

print(parser)

In [ ]:
parser.defs['variables']

In [12]:
parser.defs['variables']['mini_mnist_weights_dram'][0]

[0.02730494923889637,
 0.029643140733242035,
 -0.008366874419152737,
 0.032807547599077225,
 -0.007825127802789211,
 0.007206810638308525,
 -0.01738768070936203,
 0.020974380895495415,
 0.03148367628455162,
 -0.026201006025075912,
 0.03104272112250328,
 0.006684260908514261,
 0.02638603188097477,
 0.004836806561797857,
 0.017221003770828247,
 -0.005042536184191704,
 0.027531638741493225,
 0.005278890021145344,
 -0.016672855243086815,
 0.009103511460125446,
 -0.01645488105714321,
 -0.004188316408544779,
 -0.014505655504763126,
 0.023691821843385696,
 -0.028191793709993362,
 -0.016464656218886375,
 -0.01008481252938509,
 -0.021474016830325127,
 0.0033708258997648954,
 -0.03527425602078438,
 0.032253898680210114,
 -0.03033815324306488,
 0.027572408318519592,
 0.005943541415035725,
 -0.011596591211855412,
 0.022069642320275307,
 0.005566099192947149,
 0.028855836018919945,
 0.0039042746648192406,
 -0.011263328604400158,
 0.009595845825970173,
 -0.009684979915618896,
 0.015030627138912678,
